In [22]:
import numpy as np
import pandas as pd
from sklearn import metrics
import collections

In [123]:
def report(label, prediction_prob):
    prediction_argmax = np.argmax(prediction_prob, 1)
    print metrics.classification_report(label, prediction_argmax)
    print "Log Loss:", metrics.log_loss(label, prediction_prob, eps=1e-15)
    print

In [3]:
def convert_to_matrix(df, label_idx, prob_idx_start, prob_idx_end):
    if label_idx != None:
        labels = df.ix[:,label_idx].as_matrix()
    else:
        labels = None
    prediction_prob = df.ix[:,prob_idx_start:prob_idx_end].as_matrix()
    return labels, prediction_prob

In [101]:
def make_submission_df(names, probabilities):
    df = pd.DataFrame(data=probabilities,
                      index=names,
                      columns=['Type_1', 'Type_2', 'Type_3'])
    df.index.name = 'image_name'
    
    df['indexNumber'] = [int(i.split('.')[0]) for i in df.index]
    df.sort_values(['indexNumber'], ascending = [True], inplace = True)
    df.drop('indexNumber', 1, inplace = True)
    
    df.drop_duplicates(inplace = True)
    
    return df

In [133]:
def make_submission(csv_loc, submission_output_loc = None): 
    test_df = pd.read_csv(test_csv)
    name_idx = 1; prob_idx_start = 5; prob_idx_end = 8
    nm, pp_t = convert_to_matrix(test_df, name_idx, prob_idx_start, prob_idx_end)
    sub = make_submission_df(nm, pp_t)
    if submission_output_loc != None:
        sub.to_csv(submission_output_loc)
    return sub

In [124]:
def make_report(csv_loc):
    print "Classification Report:", csv_loc
    label_idx = 3; prob_idx_start = 5; prob_idx_end = 8
    train_df = pd.read_csv(csv_loc)
    lbl, pp = convert_to_matrix(train_df, label_idx, prob_idx_start, prob_idx_end)
    report(lbl, pp)

In [115]:
csv_dir = 'predictions/inception_v3_2_all/'
test_csv = csv_dir + 'test.csv'
output = csv_dir + 'submission_inception_v3_2.csv'
make_submission(test_csv, output)

In [125]:
train_all_csv = csv_dir + 'train.csv'
train_csv = 'predictions/inception_v3_2/train.csv'
make_report(train_all_csv)
make_report(train_csv)

Classification Report: predictions/inception_v3_2_all/train.csv
             precision    recall  f1-score   support

          0       0.68      0.21      0.32       250
          1       0.68      0.81      0.74       793
          2       0.68      0.69      0.68       457

avg / total       0.68      0.68      0.65      1500

Log Loss: 0.762723509778

Classification Report: predictions/inception_v3_2/train.csv
             precision    recall  f1-score   support

          0       0.44      0.08      0.13       250
          1       0.55      0.92      0.69       793
          2       0.61      0.16      0.25       457

avg / total       0.55      0.55      0.46      1500

Log Loss: 1.11147171792



In [130]:
leak_name = ['378.jpg', '230.jpg', '61.jpg', '258.jpg', '500.jpg', '385.jpg', '322.jpg',
             '141.jpg', '149.jpg', '422.jpg', '380.jpg', '120.jpg', '14.jpg', '2.jpg', 
             '369.jpg', '308.jpg', '186.jpg', '434.jpg', '234.jpg', '289.jpg', '178.jpg',
             '432.jpg']
leak_label = [2, 1, 2, 3, 2, 2, 3, 2, 2, 3, 2, 3, 3, 2, 2, 2, 2, 1, 2, 2, 1, 3]

In [134]:
sub = make_submission(test_csv)
name_idx = [int(i.split('.')[0]) for i in leak_name]

In [156]:
df = sub.iloc[name_idx].copy()
df['labels'] = leak_label
df['indexNumber'] = [int(i.split('.')[0]) for i in df.index]
df.sort_values(['indexNumber'], ascending = [True], inplace = True)
df.drop('indexNumber', 1, inplace = True)
df

,Type_1,Type_2,Type_3,labels
image_name,,,,
2.jpg,0.269184,0.655147,0.075669,2
14.jpg,0.048228,0.493375,0.458396,3
61.jpg,0.090551,0.900680,0.008768,2
120.jpg,0.280826,0.699566,0.019608,3
141.jpg,0.165202,0.607221,0.227577,2
149.jpg,0.107680,0.363412,0.528908,2
178.jpg,0.039353,0.443570,0.517076,1
186.jpg,0.049509,0.852027,0.098464,2
230.jpg,0.031339,0.911800,0.056860,1


In [191]:
# sub.iloc[name_idx].where(sub.iloc[name_idx]['Type_1'] > 0.01)
sub = make_submission(test_csv)
s = pd.Series(leak_label)
dummy = pd.get_dummies(s)
print dummy.shape
print sub.iloc[name_idx].shape
print type(dummy)
print type(sub.iloc[name_idx])
sub.iloc[name_idx] = dummy.values
sub.to_csv(csv_dir + 'submission_inception_v3_2_L.csv')
# s.columns
# [i == 1 for i in leak_label]
# sub.iloc[name_idx].where(sub.iloc[name_idx]['Type_1'] == [i == 1 for i in leak_label])

(22, 3)
(22, 3)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
